# Applied Data Science Capstone Project



# Extending Pharmacy Business in Hamburg


# 1. introduction

The project consists of supporting the stakeholders of a small Pharmacy franchise to extend their business in Hamburg. By leveraging the Foursquare location data, Hamburg neighborhood will be investigated based on their similar characteristics and availability of drugstore. 
 
Besides, available government data on the Hamburg population by districts, age, and gender will support the selection of the most promising region for the new business, since the proportion of the people using prescribed medicines typically increase with age.

# 2. Data Description

Two sets of data will be required for this project. The first set of data consists of the information of Hamburg districts and their geographical location. Such information is available at  https://www.geonames.org/postal-codes/DE/HH/hamburg.html. 

These data will be scraped and combined with the population data provided by the statistic office of Hamburg (Statistisches Amt für Hamburg und Schleswig-Holstein). The population data is available at https://www.statistik-nord.de/fileadmin/Dokumente/Statistische_Berichte/bevoelkerung/A_I_S_1_j_H/A_I_S1_j18.pdf. 

Finally, the districts of Hamburg will be clustered based on their similarities using Foursquare location data and the most promising district will be selected based on their population information, as well as, availability of pharmacies.

# 3. Data scrapping


## 3.1 Importing libraries 
First the libraries required for data scrapping and processing are imported

In [1]:
#importing required libraries

import pandas as pd # library to process data as dataframes
from pandas.io.html import read_html

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


## 3.2 Hamburg data



### 3.2.1 Importing hamburg geo data 
The information list of hamburg districts was imported from https://www.geonames.org/postal-codes/DE/HH/hamburg.html

In [2]:
#importing table from website
path = 'https://www.geonames.org/postal-codes/DE/HH/hamburg.html'

restable = read_html(path,  attrs={"class":"restable"})
print ("Extracted {num} restabl".format(num=len(restable)))

Extracted 1 restabl


In [3]:
#print table head
df=restable[0]
df.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3,Admin4
0,1.0,Hamburg,20095,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
1,NaN,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10
2,2.0,Hamburg,20097,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
3,NaN,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019
4,3.0,Hamburg,20099,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"


In [4]:
# assining pair indexes to df1, which contain information on hamburg neighborhoods
df1 = df[::2] 
df1.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3,Admin4
0,1.0,Hamburg,20095,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
2,2.0,Hamburg,20097,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
4,3.0,Hamburg,20099,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
6,4.0,Hamburg,20146,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"
8,5.0,Hamburg,20148,Germany,Hamburg,NaN,"Hamburg, Freie und Hansestadt","Hamburg, Freie und Hansestadt"


In [5]:
# rows containing only hamburg were deleted since it reflects only Hamburg latitude and longitue, not assigned for a neighborhood
df2 = df1.iloc[97:401, 0:3]
df2.reset_index(inplace=True)
df2.drop(['index', 'Unnamed: 0'], axis=1, inplace=True)
df2.replace(regex=['Hamburg'], value=' ', inplace=True)
df2.head()

,Place,Code
0,Harburg,21073
1,Harburg,21079
2,Tonndorf,22045
3,Uhlenhorst,22085
4,,22115


In [6]:
# assining pair indexes to df1, which contain information on hamburg neighborhoods
df3 = df[1::2] 
df3.reset_index(inplace=True)
df3.head()

,index,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3,Admin4
0,1,NaN,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10,53.552/10
1,3,NaN,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019,53.548/10.019
2,5,NaN,53.558/10.011,53.558/10.011,53.558/10.011,53.558/10.011,53.558/10.011,53.558/10.011,53.558/10.011
3,7,NaN,53.567/9.98,53.567/9.98,53.567/9.98,53.567/9.98,53.567/9.98,53.567/9.98,53.567/9.98
4,9,NaN,53.567/9.998,53.567/9.998,53.567/9.998,53.567/9.998,53.567/9.998,53.567/9.998,53.567/9.998


In [7]:
# cleaning the latitude and longitude data
df4 = df3.iloc[97:200, 0:3]
df4.reset_index(inplace=True)
df4.drop(['level_0', 'index', 'Unnamed: 0'], axis=1, inplace=True)
df4.head()
df4[['latitude','longitude']] = df4.Place.str.split(pat ='/', expand=True)
del df4['Place']
df4.head()

,latitude,longitude
0,53.459,9.981
1,53.459,9.981
2,53.587,10.117
3,53.571,10.019
4,53.528,10.147


In [8]:
#merging the latitude-longitude dataframe (df4) with the Neighborhood-postal code dataframe (df2)
df5 = pd.concat([df2, df4.reindex(df2.index)], axis=1) #merging the dataframes

df5.rename(columns={'Place': 'Neighborhood', 'Code': 'Postal Code', 'latitude': 'Latitude', 'longitude': 'Longitude'}, inplace=True)

#entering missing information
df5.iloc[4,0] = 'Billstedt'
df5.iloc[5,0] = 'Langenhorn'

#correction german characters
df5.replace(regex=['Ã¼'], value='ü', inplace=True)
df5.replace(regex=['Ã¶'], value='ö', inplace=True)
df5.replace(regex=['Ã'], value='ß', inplace=True)

df5

,Neighborhood,Postal Code,Latitude,Longitude
0,Harburg,21073,53.459,9.981
1,Harburg,21079,53.459,9.981
2,Tonndorf,22045,53.587,10.117
3,Uhlenhorst,22085,53.571,10.019
4,Billstedt,22115,53.528,10.147
5,Langenhorn,22417,53.667,10.034
6,Ottensen,22765,53.555,9.918
7,Altstadt,20095,53.545,9.995
8,Eimsbüttel,20144,53.572,9.95
9,Eppendorf,20249,53.589,9.981


### 3.2.1 Importing hamburg population data

Scrapping the Population data provided by the statistic office of Hamburg (Statistisches Amt für Hamburg und Schleswig-Holstein). The population data is available at https://www.statistik-nord.de/fileadmin/Dokumente/Statistische_Berichte/bevoelkerung/A_I_S_1_j_H/A_I_S1_j18.pdf.

In [9]:
#reading the data into a pd df
df_pop = pd.read_excel('https://www.statistik-nord.de/fileadmin/Dokumente/Statistische_Berichte/bevoelkerung/A_I_S_1_j_H/A_I_S1_j18.xlsx', 
                   'T3_1',
                   header=3)

df_pop.drop(0, inplace=True)
df_pop.drop(1, inplace=True)

df_pop = df_pop.rename(columns={'Unnamed: 0': 'Neighborhood', 'Unnamed: 1': 'Gender', 'Unnamed: 2': 'Total', '65 und \nälter': 'Older than 65'})
df_pop.head(15)

,Neighborhood,Gender,Total,0 – 3,3 – 6,6 – 12,12 – 18,18 – 20,20 – 30,30 – 45,45 – 60,60 – 65,Older than 65
2,Hamburg,männl.,1197.0,45.0,38.0,26.0,30.0,11.0,187.0,411.0,284.0,50.0,115.0
3,Altstadt,weibl.,1075.0,47.0,24.0,38.0,29.0,13.0,180.0,365.0,201.0,48.0,130.0
4,NaN,zus.,2272.0,92.0,62.0,64.0,59.0,24.0,367.0,776.0,485.0,98.0,245.0
5,HafenCity,männl.,2408.0,152.0,93.0,132.0,100.0,39.0,407.0,691.0,505.0,79.0,210.0
6,NaN,weibl.,2184.0,121.0,113.0,127.0,70.0,53.0,374.0,691.0,345.0,76.0,214.0
7,NaN,zus.,4592.0,273.0,206.0,259.0,170.0,92.0,781.0,1382.0,850.0,155.0,424.0
8,Neustadt,männl.,6761.0,202.0,155.0,195.0,200.0,77.0,940.0,2066.0,1673.0,341.0,912.0
9,NaN,weibl.,6159.0,190.0,128.0,194.0,185.0,82.0,998.0,1878.0,1281.0,276.0,947.0
10,NaN,zus.,12920.0,392.0,283.0,389.0,385.0,159.0,1938.0,3944.0,2954.0,617.0,1859.0
11,St.Pauli,männl.,11934.0,322.0,302.0,485.0,422.0,161.0,1746.0,3852.0,2974.0,528.0,1142.0


Data scrapping to be concluded during the Week5

Plan B: getting Hamburg geographical coordinates

In [10]:
address = 'Hamburg, HH'

geolocator = Nominatim(user_agent="Hamburg")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Hamburg are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Hamburg are 53.5504683, 9.99464.


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = ()

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Hamburg, HH'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]